# Agent Development
For making the actual itinerary reccomendations, we want to build a conversational agent, which handles querying the formatting a query to the graph database, selecting the most appropriate events based on the user's query and the reccomendation scores, and then writing a helpful output message explaining the logic they used to reach their conclusion. To do this, we will need an agent that requires three iterations per user query:
1. **Iteration 1**: Format a Graph DB query based on the user's input. The results of this query should return the top 10 locations of a certain type in a given city.
2. **Iteration 2**: Anaylze the results of the graph query, select the 3-5 most appropriate locations/activites to suggest to the user. Format these activities as events, with a `title`, `url`, `start_date`, and `end_date`.
3. **Iteration 3**: Review the results of the event generation, format a friendly message to the user explaining the agent's choices and asking for any additional information that might be needed.

As a general fallback, the agent should not reccomend any activites if the user's query cannot be used to guide a query on the venue graph. In this event, the agent should provide a response in the first iteration, explaining what the user needs to change about their query to provide more information.


## Tool Creation

The first thing that we need to take care of to setup the agent is to create the tool objects that the agent will have access too. This will require writing python functions as well as OpenAI tool definition objects, which should be mapped to the corresponding python function. We will want to create the following tools:
1. `venue_query(city: CityName, activity: ActivityType) -> List[VenueResult]`
2. `event_formatter(venues: List[VenueResult], start_time: str, end_time: str) -> List[EventObject]`

We will first need to create the custom objects that will be used by these functions, then implement the functions themselves, then finally, create the tool definitions, adhering to the OpenAI tool schema.

In [200]:
import urllib.parse
from enum import Enum
from datetime import datetime
from typing import Dict, Any

TIME_FORMAT = r"%Y-%m-%dT%H:%M:%S"

class CityName(Enum):
    NEW_YORK = 'NYC'
    LOS_ANGELES = 'LA'
    MIAMI = 'MIAMI'
    CHICAHO = 'CHICAGO'
    SCOTTSDALE = 'SCOTTSDALE'

    @staticmethod
    def schema():
        """Return the JSON schema of this object as a stringified JSON object."""
        return {
            "type": "string",
            "enum": [
                "NYC",
                "LA",
                "MIAMI",
                "CHICAGO",
                "SCOTTSDALE"
            ]
        }

class ActivityType(Enum):
    RESTAURANT = 'Restaurant'
    NIGHTLIFE = 'Nightlife'
    MUSEUM = 'Museum'
    EXPERIENCE = 'Experience'
    SHOPPING = 'Shopping'
    AMUSEMENT_PARK = 'Amusement Park'
    RELAXATION = 'Relaxation'
    HISTORICAL_STIE = 'Historical Site'
    ACTIVITY = 'Activity'

    @staticmethod
    def schema():
        """Return the JSON schema of this object as a stringified JSON object."""
        return {
            "type": "string",
            "enum": [
                "Restaurant",
                "Nightlife",
                "Museum",
                "Experience",
                "Shopping",
                "Amusement Park",
                "Relaxation",
                "Historical Site",
                "Activity"
            ]
        }


class VenueResult:
    """VenueResult object that represents a venue result from the API."""

    relevance_score: float
    name: str
    url: str
    rating: float
    reviews: str

    def __init__(self, relevance_score: float = None, name: str = None, url: str = None, rating: float = None, reviews: str = None, **kwargs):
        """Setup the VenueResult object."""
        assert (relevance_score is not None and float(relevance_score) >= 0 and float(relevance_score) <= 1), "Invalid Relevance Score value."
        assert (name is not None and type(name) == str and len(name) > 0), "Invalid Name value."
        assert (url is not None and type(url) == str and len(url) > 0), "Invalid URL value."
        assert (rating is not None and float(rating) >= 0 and float(rating) <= 5), "Invalid Rating value."
        assert reviews is not None and type(reviews) == str, "Invalid Reviews value."

        # Set the attributes of the VenueResult object.
        self.relevance_score = relevance_score
        self.name = name
        self.url = urllib.parse.unquote(url)
        self.rating = rating
        self.reviews = reviews

    @staticmethod
    def schema():
        """Return the JSON schema of this object as a stringified JSON object."""
        return {
            "type": "object",
            "properties": {
                "relevance_score": {
                    "type": "number",
                    "minimum": 0,
                    "maximum": 1
                },
                "name": {
                    "type": "string"
                },
                "url": {
                    "type": "string"
                },
                "rating": {
                    "type": "number",
                    "minimum": 0,
                    "maximum": 5
                },
                "reviews": {
                    "type": "string"
                }
            },
            "required": ["relevance_score", "name", "url", "rating", "reviews"]
        }



    @property
    def value(self):
        """Return the VenueResult as a JSON object."""
        return {
            "relevance_score": self.relevance_score,
            "name": self.name,
            "url": self.url,
            "rating": self.rating,
            "reviews": self.reviews
        }

class VenueInformation:
    """A smaller VenueResult object that represents the venue information needed by the EventClass"""
    name: str
    url: str

    def __init__(self, name: str = None, url: str = None, **kwargs):
        """Setup the VenueInformation object."""
        assert (
            (name is not None and type(name) == str and len(name) > 0)
            and (url is not None and type(url) == str and len(url) > 0), 
            "Invalid VenueInformation object."
        )

        # Set the attributes of the VenueInformation object.
        self.name = name
        self.url = urllib.parse.unquote(url)

    @staticmethod
    def schema():
        """Return the JSON schema of this object as a stringified JSON object."""
        return {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string"
                },
                "url": {
                    "type": "string"
                },
            },
            "required": ["name", "url"]
        }

    @staticmethod
    def gaurd(value):
        """Ensure that the value is a valid VenueInformation object."""
        try:
            assert (value is not None and type(value) == dict), "Invalid VenueInformation object."
            assert (value['name'] is not None and type(value['name']) == str and len(value['name']) > 0), "Invalid Name Value"
            assert (value['url'] is not None and type(value['url']) == str and len(value['url']) > 0), "Invalid URL Value"
            return True
        except AssertionError as assertion_error:
            print(f"Invalid VenueInformation object ({assertion_error}): {value}")
            return False


class Event:
    """Event object that represents an event at a venue."""
    title: str
    url: str
    start_time: datetime
    end_time: datetime

    def __init__(self, venue: VenueInformation = None, start_time: str = None, end_time: str = None):
        """Setup the Event object."""
        assert (
            (venue is not None and type(venue) == VenueInformation)
        ), "Invalid Venue object."

        assert self.validate_datetimestring(start_time) and self.validate_datetimestring(end_time), "Invalid datetime string."

        # Set the attributes of the Event object.
        self.title = venue.name
        self.url = venue.url


        self.start_time = datetime.strptime(start_time, TIME_FORMAT)
        self.end_time = datetime.strptime(end_time, TIME_FORMAT)

    @staticmethod
    def schema():
        """Return the JSON schema of this object as a dictionary."""
        return {
            "type": "object",
            "properties": {
                "title": {
                    "type": "string"
                },
                "url": {
                    "type": "string"
                },
                "start_time": {
                    "type": "string",
                    "format": "%Y-%m-%dT%H:%M:%S"
                },
                "end_time": {
                    "type": "string",
                    "format": "%Y-%m-%dT%H:%M:%S"
                }
            },
            "required": ["title", "url", "start_time", "end_time"]
        }

    @staticmethod
    def validate_datetimestring(datetime_str: str) -> bool:
        """Validate a datetime string."""
        try:
            datetime.strptime(datetime_str, TIME_FORMAT)
            return True
        except ValueError:
            return False

    @property
    def value(self):
        """Return the Event as a JSON object."""
        return {
            "title": self.title,
            "url": self.url,
            "start_time": self.start_time.strftime(TIME_FORMAT),
            "end_time": self.end_time.strftime(TIME_FORMAT)
        }

    
    @classmethod
    def from_venue(cls, venue: VenueInformation, start_time: str = None, end_time: str = None) -> 'Event':
        """Create an Event object from a VenueResult object."""
        # Validate the input arguments.
        assert (
            (venue is not None and type(venue) == VenueInformation)
        ), "Invalid Venue object."
        assert cls.validate_datetimestring(start_time) and cls.validate_datetimestring(end_time), "Invalid datetime string."
        
        return cls(venue, start_time, end_time)


<>:132: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:132: SyntaxWarning: assertion is always true, perhaps remove parentheses?
/var/folders/18/gysclhzd7dqcgbf94d0nx0mh0000gn/T/ipykernel_5944/3871169907.py:132: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (


In [ ]:
import os
from typing import List

from neo4j import GraphDatabase
from neo4j.graph import Node

DB_USER = os.getenv("NEO4J_DATABASE_USERNAME")
DB_URL = os.getenv("NEO4J_DATABASE_URL")
DB_PASSWORD = os.getenv("NEO4J_DATABASE_PASSWORD")


def graph_driver(query: str) -> List[Node]:
    """Execute a query on the Neo4j database."""
    driver = GraphDatabase.driver(DB_URL, auth=(DB_USER, DB_PASSWORD))
    return driver


In [201]:
import random
import tiktoken

class VenueQueryTool:

    def __init__(self, city: str = None, activity_type: str = None, page_num: int = 1, **kwargs):
        """Setup the VenueQueryTool object."""
        assert (
            (city is not None and type(city) == str)
            and (activity_type is not None and type(activity_type) == str)
        ), "Invalid VenueQueryTool object."


        # Set the attributes of the VenueQueryTool object.
        try:
            self.city = CityName(city)
            self.activity_type = ActivityType(activity_type)
            self.page_num = int(page_num)
        except ValueError as e:
            raise ValueError(f"Invalid city or activity type: {city}, {activity_type}.") from e


    def __call__(self) -> List[VenueResult]:
        """Execute the VenueQueryTool."""

        # Generate 10 random venues. 
        query = (
            f"MATCH (v: Venue)-[:HAS_REVIEW]->(r: Review) "
            f"WHERE v.category = '{self.activity_type.value}' AND v.city = '{self.city.value}' AND toFloat(v.rating) >= 4 "
            f"RETURN v, COLLECT(r) as reviews "
            f"ORDER BY rand() LIMIT 10;"
        )
        driver = graph_driver(query)

        enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

        venue_data_with_reviews = []
        with driver.session() as session:
            results = session.run(query)
            for record in results:
                venue = record['v']
                reviews = record['reviews']


                # Trim the concat_reviews to only 500 tokens long
                concat_reviews = '\n'.join([f"{review.get('author')}: {review.get('text')}" for review in reviews])
                tokens = enc.encode(concat_reviews)
                if len(tokens) > 250:
                    tokens = tokens[:250] + enc.encode('...')
                concat_reviews = enc.decode(tokens)
                
                venue_data = {
                    'name': venue.get('name'),
                    'url': venue.get('url'),
                    'rating': venue.get('rating'),
                    'relevance_score': random.uniform(0, 1),
                    'reviews': concat_reviews
                }

                venue_data_with_reviews.append(venue_data)

        # Close the session once the records have been parsed
        driver.close()
            
        venues = [
            VenueResult(
                relevance_score=venue['relevance_score'], 
                name=venue['name'], 
                url=venue['url'], 
                rating=venue['rating'],
                reviews=venue['reviews']
            ) 
            for venue in venue_data_with_reviews
        ]
        venues.sort(key=lambda x: x.relevance_score, reverse=True)
        return venues

    @staticmethod
    def schema():
        return {
            "type": "function",
            "function": {
                "name": "venue_query",
                "description": "Query for relevant for a user, given a city and activity type.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": CityName.schema(),
                        "activity_type": ActivityType.schema(),
                        "page_num": {
                            "type": "integer",
                            "default": 1
                        }
                    },
                },
                "required": ["city", "activity_type"]
            }
        }

q_tool = VenueQueryTool(city='NYC', activity_type='Restaurant')
results = q_tool()
for result in results:
    print(result.value)

{'relevance_score': 0.8323274542757377, 'name': 'Monkey Bar', 'url': 'https://www.yelp.com/biz/monkey-bar-new-york-4?adjust_creative=Fj58EcsIOfJVBvvlE6xZdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Fj58EcsIOfJVBvvlE6xZdw', 'rating': '4.0', 'reviews': "Alex H: The burgers are as good as the ones from Au Cheval and 4 Charles, which means they're one of the best in the city. We had the regular burger and the wagyu one, and while I didn't think there was much of a difference, my partner thought the wagyu ones were much juicier. Regardless they're both excellent.We ordered the potato pancakes and they were disappointing. They were a little dry and the oil used tasted a lot like the ones used by McDonald's - not good. I shouldn't be comparing this place to McD's.Good service and the ambiance is very much like an old speakeasy. Burger Potato pancakes\nEmery Y: Hogsalt, so glad you made it here! This is my seventh visit to your associated restaurants (all the other 

In [202]:


class EventCreatorTool:

    def __init__(self, venues: List[VenueInformation] = None, start_time: str = None, end_time: str = None, **kwargs):
        assert venues is not None
        assert type(venues) == list
        assert len(venues) > 0
        assert all([VenueInformation.gaurd(venue) for venue in venues])
        assert Event.validate_datetimestring(start_time) and Event.validate_datetimestring(end_time), "Invalid datetime string."
        self.venues = [VenueInformation(**venue) for venue in venues]
        self.start_time = start_time
        self.end_time = end_time

    def __call__(self) -> List[Event]:
        """Execute the EventCreatorTool."""
        # We simply create an event for each venue.
        return [Event.from_venue(venue, self.start_time, self.end_time) for venue in self.venues]
        
    @staticmethod
    def schema():
        return {
            "type": "function",
            "function": {
                "name": "event_creator",
                "description": "Create events from a list of venues.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "venues": {
                            "type": "array",
                            "items": VenueInformation.schema()
                        },
                        "start_time": {
                            "type": "string",
                            "format": "%Y-%m-%dT%H:%M:%S"
                        },
                        "end_time": {
                            "type": "string",
                            "format": "%Y-%m-%dT%H:%M:%S"
                        }
                    },
                    "required": ["venues", "start_time", "end_time"]
                }
            }
        }


def execute_tool(_id: str, name: str, **kwargs):
    """Execute a tool given a name and a set of arguments."""
    if name == "venue_query":
        tool = VenueQueryTool(**kwargs)
    elif name == "event_creator":
        tool =  EventCreatorTool(**kwargs)
    else:
        raise ValueError(f"Invalid tool name: {name}.")
    result = tool()
    data = [item.value for item in result]
    return {
        "tool_call_id": _id,
        "role": "tool",
        "name": name,
        "content": json.dumps(data)
    }

tools = [VenueQueryTool.schema(), EventCreatorTool.schema()]

In [205]:
import json
from typing import Any, Tuple

from openai import OpenAI

SYSTEM_PROMPT = """
You are a helpful travel agent. You help you're customers build their dream vacation. You are conversing
with a customer about an upcoming trip. You are trying to help them find additional activites to do while they 
are on their trip.

You're customer will provide you with a description about a specific event that would like to do on their trip. Your
task is to find potential locations that they can visit/make reservations at for that activity. You are provided with a 
querying tool, that will allow you to search for locations by city and activity type. The results of thsi query will
be ranked in regards to their correlation with the customer's social media profile. The higher the correlation, the more
likely the customer will be interested in visiting that location.

Once you have received the ranked list of results, you then must determine if the results returned to you will satisfy the user's request.
Read through some of the provided reviews for each location, and make a determination about whether or not the location is a good suggestion,
based on the user's request. If you cannot find 3-5 locations from the results that you think are a good fit, then you may query for more events,
by using the page argument of the querying tool.

For example, suppose the user requests a breakfast location. You should read the reviews to first determine if each location seems like they serve
breakfast, and then you should think about weather or not the reviews indicate that the location is a good fit for the user, based on their request.
You should only suggest a location if you are INCREDIBLY CONFIDENT that it will satisfy exactly what the user is looking for. If it is not very clear
that a location will satisfy the user's needs, it should not be suggested.

Once you have your locations, you will have access to an event builder, that allows you to create potential calendar events for the potential
locations that you suggest to the customer. You will use this tool to create a formatted list of suggestions for your user. Pass the 
3-5 locations that you have selected to the event builder, along with the start and end time of the event.

Finally, once you have created the formatted list of calendar events, you will write a short response to the customer, which will
provide a breif justification for why you think the suggested locations are a good fit for the customer (1-2 sentences).

After you're response has been received by the customer, they can either accept or reject your suggestions. If they reject them, they
will give you more guidance about what they are looking for. When you receive more guidance from a customer, you will follow the same
three step process to provide them with a new list of suggestions. 

If, at any point during the conversation, the customer sends you a message that does not make sense, or cannot be used to guide you to
create event suggestions, you should respond directly to the customer with a message, that instructs them on what information they must
provide you with in order for you to assist them.
"""

class Agent:

    def __init__(self, model: str = 'gpt-3.5-turbo-1106'):
        """Setup the agent object."""
        self._tools = tools
        self.messages = [
            {'role': 'system', 'content': SYSTEM_PROMPT}
        ]
        self.model = model
        self.client = OpenAI()
        self._finish_reason = None

    def __call__(self, query: str) -> Tuple[str, List[Event]]:
        """Execute the agent."""
        self.messages.append({'role': 'user', 'content': query})
        

        count = 0
        while count < 5:
            # Execute a single step of the agent.
            self._step()

            # Get the last message from the agent (the result of the last step)
            last_message = self.messages[-1]

            # If the last message is a tool_call, execute the tool call.
            if self._finish_reason == 'tool_calls' and len(last_message.tool_calls) > 0:
                tool_calls = last_message.tool_calls

                # If the agent is producing multiple calls per step, that is a problem.
                assert len(tool_calls) == 1, "Only one tool call can be made at a time."

                tool_call = tool_calls[0]
                self._execute_tool(tool_call)

            # Otherwise, if the last message has content, then we may return the results.
            if self._finish_reason == 'stop':
                break
            
            # Increment the count to track iteratins
            count += 1
        
        return self.messages[-1].content

        
    def _step(self) -> None:
        """Execute a single step of the agent."""
        completion = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            tools=self._tools,
            tool_choice='auto',
            temperature=0.5
        )
        msg = completion.choices[0].message
        self._finish_reason = completion.choices[0].finish_reason
        self.messages.append(msg)

    def _execute_tool(self, tool_call_message: Any) -> None:
        """Execute a tool call."""
        tool_call_id = tool_call_message.id
        tool_name = tool_call_message.function.name
        tool_args = json.loads(tool_call_message.function.arguments)
        print(f"({tool_call_id}) Making tool call: '{tool_name}' with: {tool_args}")
        tool_result = execute_tool(tool_call_id, tool_name, **tool_args)
        print(f"({tool_call_id}) Tool call result:\n{tool_result}\n")
        self.messages.append(tool_result)
        

agent = Agent()
agent("I need help finding a breakfast location for my tuesday morning trip to New York City.")


(call_xrvtIbPT2iCK7TlXyQzHmO2h) Making tool call: 'venue_query' with: {'city': 'NYC', 'activity_type': 'Restaurant'}
(call_xrvtIbPT2iCK7TlXyQzHmO2h) Tool call result:
{'tool_call_id': 'call_xrvtIbPT2iCK7TlXyQzHmO2h', 'role': 'tool', 'name': 'venue_query', 'content': '[{"relevance_score": 0.9541585065477316, "name": "Misi", "url": "https://www.yelp.com/biz/misi-brooklyn?adjust_creative=Fj58EcsIOfJVBvvlE6xZdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Fj58EcsIOfJVBvvlE6xZdw", "rating": "4.0", "reviews": "Jess W: Misi is pasta (and literally everything else) done right. Immediately upon entering, you\'re enveloped in a special kind of energy...the kind of buzz and excitement where you feel like you\'re in the presence of something special about to happen. And that something special comes in the form of ricotta toast and seasonal pastas.You\'ve probably seen the ricotta toast everywhere, and I\'m here to tell you that YES it tastes as rich and luscious as it look

"I have found some great breakfast locations for your Tuesday morning trip to New York City:\n\n1. [Misi](https://www.yelp.com/biz/misi-brooklyn?adjust_creative=Fj58EcsIOfJVBvvlE6xZdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Fj58EcsIOfJVBvvlE6xZdw) - Enjoy delicious ricotta toast and seasonal pastas.\n2. [ARIARI](https://www.yelp.com/biz/ariari-new-york?adjust_creative=Fj58EcsIOfJVBvvlE6xZdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Fj58EcsIOfJVBvvlE6xZdw) - Experience unique and creative Korean dishes in a stylish atmosphere.\n3. [Blue Park Kitchen](https://www.yelp.com/biz/blue-park-kitchen-new-york?adjust_creative=Fj58EcsIOfJVBvvlE6xZdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Fj58EcsIOfJVBvvlE6xZdw) - Enjoy customizable bowls with fresh and wholesome ingredients.\n\nThese locations offer a variety of breakfast options and have received great reviews. Let me know if you need more information or if 

## Conclusions

After setting up the agent, it is clear that the agent needs a more effective way of narrowing down the results, when it writes a query. One possible way to do this would be to setup a vector index. The vector index would be used by the agent to write a natural language query that would return all locations that satisfy the discrete filters and they would be sorted by relevance. To do this, we could embed the reviews of associated with each location. Then, we could create a vector store with each location. The vector store would have the business ID stored in the metadata, along with the city and category (both as indexes). This way, we can first query the vector store, and get a list of general venues that are somewhat inline with what the user is looking for. Once that query has resolved, we will then use the Yelp business ID's of each result, to apply a filter on the query on the graph database. This way, we will be querying over the space of all venues that met a certain similarity score with the user's query and relating that to the space of the user's posts. 